In [1]:
import pydicom
from glob import glob
from torch.utils.data import Dataset
from skimage import transform
import sklearn.preprocessing as skp
import numpy as np
import torch
from typing import Tuple

In [2]:
data_path = "/Users/abinavr/Desktop/mri_data/train/**/**/T1postcontrast"
label_path = "/Users/abinavr/Desktop/mri_data/train/**/**/T2SPACE"

In [3]:
def to_tensor(array):
    """
    input image array of form WxHxD
    output tensor of the form DxHxW
    """
    img = np.transpose(array, (2, 0, 1))
    img = torch.from_numpy(img)
    return img


def normalize(array):
    """
    Input image is given as a 2D array
    Resizing is done with sklearn.preprocessing.normalize
    """
    image = skp.normalize(array[:, :, 0])
    return image

def resize(array, size):
    """Resizes a given array into size shape

    Args:
        array (numpy.ndarray): Image to be resized
        size (Tuple): Size of the output image
    """
    image = transform.resize(array, size)
    return image

In [67]:
class DataLoader(Dataset):
    def __init__(self, dataset_path, label_path,size):
        self.dataset = glob(dataset_path+"*.dcm", recursive=True)
        self.labels = glob(dataset_path+"*.dcm", recursive=True)
        self.image_size = (size, size)
        
    def __len__(self)-> int:
        return len(self.dataset)

    def __getitem__(self, index):
        print(self.dataset[index])
        pic = pydicom.dcmread(self.dataset[index]).pixel_array
        resized_pic = resize(pic, self.image_size)
        normalized_pic = np.expand_dims(resized_pic, axis=-1)
        data = to_tensor(normalized_pic)
        return data

In [9]:
class ImageLoader(Dataset):
    def __init__(self, dataset_path: str, label_path: str, image_size: int) -> None:
        super(ImageLoader, self).__init__()
        self.dataset = glob(f"{dataset_path}/*.dcm")
        self.labels = glob(f"{label_path}/*.dcm")
        self.image_size: Tuple = (image_size, image_size)

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, index):
        print(self.dataset[index], self.labels[index])
        pic = pydicom.dcmread(self.dataset[index]).pixel_array
        label = pydicom.dcmread(self.labels[index]).pixel_array
        resized_pic = resize(pic, self.image_size)
        resized_label = resize(label, self.image_size)
        normalized_pic = np.expand_dims(resized_pic, axis=-1)
        label = np.expand_dims(resized_label, axis=-1)
        data = to_tensor(normalized_pic)
        label = to_tensor(label)
        return data, label


In [10]:
train_dataloader = ImageLoader(dataset_path=data_path,label_path=label_path, image_size=192)

In [11]:
len(train_dataloader)

14194

In [15]:
for data,label in train_dataloader:
    print(data.shape, label.shape)

/Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T1postcontrast/1-013.dcm /Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T2SPACE/1-013.dcm
torch.Size([1, 192, 192]) torch.Size([1, 192, 192])
/Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T1postcontrast/1-007.dcm /Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T2SPACE/1-007.dcm
torch.Size([1, 192, 192]) torch.Size([1, 192, 192])
/Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T1postcontrast/1-167.dcm /Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T2SPACE/1-167.dcm
torch.Size([1, 192, 192]) torch.Size([1, 192, 192])
/Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T1postcontrast/1-173.dcm /Users/abinavr/Desktop/mri_data/train/ReMIND-071/12-25-1982-NA-Preop-44230/T2SPACE/1-173.dcm
torch.Size([1, 192, 192]) torch.Size([1, 192, 192])
/Users/abinavr/Deskt

ValueError: axes don't match array